In [ ]:
#Language processing RNN
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt
#Data preparation
#shakespeare_url = "https://homl.info/shakespeare" # shortcut URL
#filepath = keras.utils.get_file("shakespeare.txt", shakespeare_url)
filepath='/Users/liaoyiwei/.keras/datasets/shakespeare.txt'
with open(filepath) as f:
    shakespeare_text=f.read()

In [ ]:
Tokenizer=keras.preprocessing.text.Tokenizer(char_level=False)
Tokenizer.fit_on_texts([shakespeare_text])
max_id=len(Tokenizer.word_index)
[encoded]=np.array(Tokenizer.texts_to_sequences([shakespeare_text]))-1
data_size=len(encoded)

train_size=data_size*9//100

In [ ]:
train_size

In [ ]:
data_train=tf.data.Dataset.from_tensor_slices(encoded[:train_size])
dataTrain=data_train.window(100,1,drop_remainder=True)
dataTrain = dataTrain.flat_map(lambda window: window.batch(101))

In [ ]:
dataset=dataTrain.shuffle(10000).batch(32)
dataset=dataset.map(lambda window: (window[:,:-1],window[:,1:]))
dataset = dataset.map(lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset=dataset.prefetch(1)

In [ ]:
model = keras.models.Sequential([
 keras.layers.Input(shape=[None,max_id]),
 keras.layers.GRU(128, return_sequences=True, 
 dropout=0.2, recurrent_dropout=0.2),
 keras.layers.GRU(128, return_sequences=True,
 dropout=0.2, recurrent_dropout=0.2),
 keras.layers.TimeDistributed(keras.layers.Dense(max_id,
 activation="softmax"))
])


In [ ]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='sgd')

In [ ]:
model.fit(dataset,epochs=20)